In [1]:
import numpy as np 
import scipy as sp 
import pandas as pd 

import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from scipy.stats import norm  
np.random.seed(123)

sns.set_style('white')
sns.set_context('talk')

from sklearn.preprocessing import StandardScaler

import scipy.special as sp

from sklearn import metrics

from scipy.stats import multivariate_normal

from scipy.stats import shapiro
from scipy.stats import anderson

from sklearn.utils import shuffle

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from sklearn import svm

from sklearn import naive_bayes

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_val_predict

3, 5, 7, 9, 10, 13

In [2]:
def convert_bad_data(Data):
    Data_list = []
    for string in Data:
        Data_list.append(np.array([float(i) for i in str(string[0]).split()]))
    
    return np.array(Data_list)

In [3]:
def my_plot_importance(booster, figsize, **kwargs):
    from xgboost import plot_importance
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax, **kwargs)

In [4]:
def asy1(estimator, X, y):
    Penalty_1 = np.array([[-10,10], [1,0]])
    answ = np.int64(estimator.predict(X))
    return np.sum(Penalty_1[y, answ])
    
def asy2(estimator, X, y):
    Penalty_2 = np.array([[-1,2], [1,-1]])
    answ = np.int64(estimator.predict(X))
    return np.sum(Penalty_2[y, answ])

def num(estimator, X, y):
    answ = np.int64(estimator.predict(X))
    return np.sum(np.abs(answ - y) > 0)

def _num(y1, y2):
    return np.sum(np.abs(y1 - y2) > 0)

def _acc(y1, y2):
    return np.sum(y1 == y2)

def _asy1(y1, y2):
    Penalty_1 = np.array([[-10,10], [1,0]])
    return np.sum(Penalty_1[y1, y2])

def _asy2(y1, y2):
    Penalty_2 = np.array([[-1,2], [1,-1]])
    return np.sum(Penalty_2[y1, y2])

In [5]:
def comp(X, lower, upper):
    outliers = []
    for i in range(X.shape[0]):
        if np.sum(X[i] < lower) + np.sum(X[i] > upper) > 0:
            outliers.append(i)
    return outliers
 
def check_outliers(X):
    q25 = X.quantile(0.25, axis=0)
    q75 = X.quantile(0.75, axis=0)
    iqr = q75 - q25
    cut_off = iqr * 1.75
    lower, upper = q25 - cut_off, q75 + cut_off
    outliers = comp(X.values, lower, upper)
    return outliers

# Dataset № 13

In [6]:
#X_train = convert_bad_data(pd.read_csv('./data/task1_13_learn_X.csv').values)
#y_train = np.int64(np.reshape(pd.read_csv('./data/task1_13_learn_y.csv').values, -1))

#X_test = convert_bad_data(pd.read_csv('./data/task1_13_test_X.csv').values)

X_train = pd.read_csv('./data/task1_13_learn_X.csv', header=None, sep=' ').values
y_train = np.reshape(np.int64(pd.read_csv('./data/task1_13_learn_y.csv',header=None, sep=' ').values), -1)

X_test = pd.read_csv('./data/task1_13_test_X.csv',header=None, sep=' ').values

In [7]:
df = pd.DataFrame(X_train)

In [8]:
X_train.shape, y_train.shape, X_test.shape

((1000, 10), (1000,), (10000, 10))

In [9]:
# df.info()

In [10]:
# df.head()

In [11]:
# df.describe()

In [12]:
print("0 in train:", X_train[y_train==0].shape[0], '\n', "1 in train:", X_train[y_train==1].shape[0])

0 in train: 834 
 1 in train: 166


In [13]:
# df.corr()

In [14]:
X_scaled = X_train#StandardScaler(with_mean=False).fit_transform(X_train)

#df.apply(average)
#df.apply(max) - df.apply(min)

In [15]:
df_normalized = pd.DataFrame(X_scaled)

In [16]:
# plt.figure(figsize=(15,8))
# sns.heatmap(df_normalized.corr(), annot=True)

In [17]:
# df_normalized.head()

In [18]:
# df_normalized.describe()

In [19]:
df_selected_features = df_normalized[[0, 2, 4, 6, 8, 9]]

In [20]:
# plt.figure(figsize=(15,8))
# sns.heatmap(df_selected_features.corr(), annot=True)

In [21]:
df = pd.concat([df_selected_features, (pd.DataFrame(y_train)).rename(columns={0: 10})], axis=1)

In [22]:
# Deleted_parameters = check_outliers(pd.DataFrame(X_train))
# X_train_ = np.delete(X_train, Deleted_parameters, axis = 0)
# y_train = np.delete(y_train, Deleted_parameters, axis = 0)

In [23]:
#sns.pairplot(pd.concat([df_normalized, pd.DataFrame(y_train)], axis=1, ignore_index=True), hue=10)
# sns.pairplot(df, hue=10)

In [24]:
df.columns

Int64Index([0, 2, 4, 6, 8, 9, 10], dtype='int64')

In [25]:
# _=plt.hist(X_scaled[np.where(y_train==1)][:,1], bins=20, color=(1,0,0,0.5))
# _=plt.hist(X_scaled[np.where(y_train==0)][:,1], bins=20, color=(0,0,1,0.5))

In [26]:
# plt.plot(X_scaled[np.where(y_train==1)].mean(0))
# plt.plot(X_scaled[np.where(y_train==0)].mean(0))

In [27]:
svm_ = svm.OneClassSVM(gamma=10, nu=0.1)
#data = pandas.read_csv("train.csv", na_values="NaN")
sdata = shuffle(df, random_state=321)

svm_.fit(sdata)
labels = svm_.predict(sdata)

In [28]:
(labels==1).mean() # -- не выбросы

0.40000000000000002

In [38]:
print(sdata.shape, y_train.shape)

(1000, 7) (1000,)


In [29]:
# sns.pairplot(df[labels==1], hue=10)

In [30]:
# values = sdata.values
norm_obj = [i for i in range(labels.shape[0]) if labels[i]==1] # Номера объектов, не являющихся выбросами
# anomaly_obj = [i for i in range(labels.shape[0]) if labels[i]==-1] # Номера шумовых объектов
# n_features = sdata.shape[1] - 1
# features = sdata.columns.values[:-1]
# x = y = 0

# fig, axes = plt.subplots(2, 3, figsize = (12,8))
# for i in range(n_features):
#     for j in range(i+1, n_features):
#         axes[x, y%3].scatter(values[anomaly_obj, i], values[anomaly_obj, j], c="red")
#         axes[x, y%3].scatter(values[norm_obj, i], values[norm_obj, j], c="blue")
#         axes[x, y%3].set_xlabel(features[i])
#         axes[x, y%3].set_ylabel(features[j])
#         y += 1
#     if i == 0:
#         x += 1

In [31]:
# values = sdata.values
norm_obj = [i for i in range(labels.shape[0]) if labels[i]==1] # Номера объектов, не являющихся выбросами
# anomaly_obj = [i for i in range(labels.shape[0]) if labels[i]==-1] # Номера шумовых объектов
# n_features = sdata.shape[1] - 1
# features = sdata.columns.values[:-1]
# x = y = 0

# fig, axes = plt.subplots(2, 3, figsize = (12,8))
# for i in range(n_features):
#     for j in range(i+1, n_features):
#         axes[x, y%3].scatter(values[anomaly_obj, i], values[anomaly_obj, j], c="red")
#         axes[x, y%3].scatter(values[norm_obj, i], values[norm_obj, j], c="blue")
#         axes[x, y%3].set_xlabel(features[i])
#         axes[x, y%3].set_ylabel(features[j])
#         y += 1
#     if i == 0:
#         x += 1

In [32]:
svm_filtered_df = df[labels==1]

In [33]:
print("0 in train:", svm_filtered_df[svm_filtered_df[10]==0].shape[0], '\n', "1 in train:", svm_filtered_df[svm_filtered_df[10]==1].shape[0])

0 in train: 335 
 1 in train: 65


In [34]:
def replace_outliers(X, alpha):
    num_features = X.shape[1]
    q_low = X.quantile(alpha, axis=0)
    q_up = X.quantile(1 - alpha, axis=0)
    X_new = X.values.copy()
    print(num_features, X.shape[0])
    
    for i in range(num_features):
        median = X[i].median()
    for j in range(X.shape[0]):
        if X_new[j][i] < q_low[i] or X_new[j][i] > q_up[i]:
            X_new[j][i] = median
            
    return X_new

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
#rho = 0.9
#Predict = cross_val_predict(estimator=naive_bayes.GaussianNB(priors=[rho,1-rho]), X=X_train_new, y=y_train_new, cv = 10, method='predict_proba')
Predict = cross_val_predict(estimator=LogisticRegression(penalty = 'l1'), X=svm_filtered_df.values, y=y_train[norm_obj], cv = 10, method='predict_proba')

print('ACC', metrics.accuracy_score(y_train[norm_obj], np.argmax(Predict, axis = 1)))
print('AUC', metrics.roc_auc_score(y_train[norm_obj], Predict[:,1]))
print('NUM', _num(y_train[norm_obj], np.argmax(Predict, axis = 1)))
print('ASY1', _asy1(y_train[norm_obj], np.argmax(Predict, axis = 1)))
print('ASY2', _asy2(y_train[norm_obj], np.argmax(Predict, axis = 1)))

#Predict_For_ASY1 = naive_bayes.GaussianNB(priors=[rho,1-rho]).fit(X_train, y_train).predict(X_test)
#Predict_For_ASY1 = naive_bayes.GaussianNB(priors=[rho,1-rho]).fit(X_train, y_train).predict(X_test)

ACC 1.0
AUC 1.0
NUM 0
ASY1 -3350
ASY2 -400


In [37]:
# cross_val_predict(estimator=LogisticRegression(penalty = 'l1'), X=svm_filtered_df.values, y=y_train[norm_obj], cv = 10, method='predict_proba')